In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creat-50candidates/pred_df_carts.pqt
/kaggle/input/creat-50candidates/pred_df_buys.pqt
/kaggle/input/creat-50candidates/__results__.html
/kaggle/input/creat-50candidates/pred_df_clicks.pqt
/kaggle/input/creat-50candidates/__notebook__.ipynb
/kaggle/input/creat-50candidates/__output__.json
/kaggle/input/creat-50candidates/custom.css
/kaggle/input/otto-recommender-system/sample_submission.csv
/kaggle/input/otto-recommender-system/test.jsonl
/kaggle/input/otto-recommender-system/train.jsonl
/kaggle/input/otto-validation/test_labels.parquet
/kaggle/input/otto-validation/test_parquet/011.parquet
/kaggle/input/otto-validation/test_parquet/006.parquet
/kaggle/input/otto-validation/test_parquet/017.parquet
/kaggle/input/otto-validation/test_parquet/016.parquet
/kaggle/input/otto-validation/test_parquet/005.parquet
/kaggle/input/otto-validation/test_parquet/019.parquet
/kaggle/input/otto-validation/test_parquet/012.parquet
/kaggle/input/otto-validation/test_parquet/015.parquet
/ka

In [2]:
train = pd.read_parquet('/kaggle/input/otto-train-and-test-data-for-local-validation/train.parquet')
#train_labels = pd.read_parquet('')
#train_labels = train_labels.explode('ground_truth')
#train_labels = train_labels.rename(columns = {'ground_truth': 'aid'})
#train_labels = train_labels.replace({'clicks':0, 'carts': 1, 'orders': 2})

In [3]:
val = pd.read_parquet('/kaggle/input/otto-train-and-test-data-for-local-validation/test.parquet')

In [4]:
train.head()

,session,aid,ts,type
0,0,1517085,1659304800,0
1,0,1563459,1659304904,0
2,0,1309446,1659367439,0
3,0,16246,1659367719,0
4,0,1781822,1659367871,0


In [5]:
val.head()

,session,aid,ts,type
0,11098528,11830,1661119200,0
1,11098529,1105029,1661119200,0
2,11098530,264500,1661119200,0
3,11098530,264500,1661119288,0
4,11098530,409236,1661119369,0


In [6]:
df_concat = pd.concat([train, val])

In [7]:
item_features = df_concat.groupby('aid').agg({'aid':'count','session':'nunique','type':'mean'})
item_features.columns = ['item_item_count','item_user_count','item_buy_ratio']
# CONVERT COLUMNS TO INT32 and FLOAT32 HERE
item_features.to_parquet('item_features.pqt')

In [8]:
import gc
del item_features, train, df_concat
_ = gc.collect()

In [9]:
user_features = val.groupby('session').agg({'session':'count','aid':'nunique','type':'mean'})
user_features.columns = ['user_user_count','user_item_count','user_buy_ratio']
# CONVERT COLUMNS TO INT32 and FLOAT32 HERE
user_features.to_parquet('user_features.pqt')

In [10]:
del user_features
_ = gc.collect()

In [11]:
val_click = val.loc[val['type'] == 0]
val_click = val_click.drop('ts', axis=1)
val_click = val_click.rename(columns = {'type': 'feature_click'})
val_click['feature_click'].replace(0, 1, inplace=True)

val_click.to_parquet('val_click.pqt')

In [12]:
val_cart = val.loc[val['type'] == 1].drop('ts', axis=1)
val_cart = val_cart.rename(columns={'type': 'feature_cart'})

val_buy = val.loc[val['type'] == 2].drop('ts', axis=1)
val_buy = val_buy.rename(columns={'type': 'feature_buy'})
val_buy['feature_buy'].replace(2, 1, inplace=True)

val_cart.to_parquet('val_cart.pqt')
val_buy.to_parquet('val_buy.pqt')

In [13]:
del val_click, val, val_cart, val_buy
_ = gc.collect()

In [14]:
click_candidates = pd.read_parquet('/kaggle/input/creat-50candidates/pred_df_clicks.pqt')

In [15]:
#click_candidates = click_candidates.explode('aids')
click_candidates

,session,aids
0,11098528,11830
0,11098528,588923
0,11098528,1732105
0,11098528,571762
0,11098528,884502
...,...,...
99999,11198527,637827
99999,11198527,485256
99999,11198527,683401
99999,11198527,544144


In [16]:
#click_candidates['session'] =click_candidates['session'].astype('int32')

In [17]:
click_candidates.dtypes

session    int32
aids       int32
dtype: object

In [18]:
item_features = pd.read_parquet('item_features.pqt')
click_candidates = click_candidates.merge(item_features, left_on='aids', right_index=True, how='left').fillna(-1)
user_features = pd.read_parquet('user_features.pqt')
click_candidates = click_candidates.merge(user_features, left_on='session', right_index=True, how='left').fillna(-1)

In [19]:
#del item_features, user_features
#_ = gc.collect()

In [20]:
candidates= click_candidates.iloc[:15000000]

In [21]:
val_click = pd.read_parquet('val_click.pqt')
val_cart = pd.read_parquet('val_cart.pqt')
val_buy = pd.read_parquet('val_buy.pqt')

In [22]:
val_click

,session,aid,feature_click
0,11098528,11830,1
1,11098529,1105029,1
2,11098530,264500,1
3,11098530,264500,1
4,11098530,409236,1
...,...,...,...
7683572,12899774,33035,1
7683573,12899775,1743151,1
7683574,12899776,548599,1
7683575,12899777,384045,1


In [23]:
val_cart

,session,aid,feature_cart
7,11098530,409236,1
34,11098533,1309900,1
35,11098533,1074173,1
59,11098535,745365,1
83,11098537,1409748,1
...,...,...,...
7683462,12899688,468148,1
7683517,12899732,1126169,1
7683527,12899739,1379999,1
7683552,12899757,1677695,1


In [24]:

CHUNKS = 10
chunk_size = len(candidates)
chunk = int(np.ceil(len(candidates) / CHUNKS))
for k in range(CHUNKS):
    
    a = k*chunk
    if a==0:
        pass
    else:
        a=b
    print(a)
    b = min( (k+1)*chunk, len(candidates) )
    print(b)
    df = candidates.iloc[a:b].copy()
    df = df.merge(val_click, left_on=['session', 'aids'],right_on=['session', 'aid'], how='left').fillna(0)
    df = df.drop('aid', axis=1)
    df = df.merge(val_cart, left_on=['session', 'aids'],right_on=['session', 'aid'] ,how='left').fillna(0)
    df = df.drop('aid', axis=1)
    df = df.merge(val_buy, left_on=['session', 'aids'],right_on=['session', 'aid'], how='left' ).fillna(0)
    df = df.drop('aid', axis=1)
    df = df.drop_duplicates()
    df.to_parquet(f'click_candidate_with_features_p{k}.pqt')

0
1500000
1500000
3000000
3000000
4500000
4500000
6000000
6000000
7500000
7500000
9000000
9000000
10500000
10500000
12000000
12000000
13500000
13500000
15000000


In [25]:
#click_candidates.duplicated(subset=['session', 'aids']).value_counts()
del click_candidates, df, val_click, val_cart, val_buy
_ = gc.collect()

In [26]:
click_tar = pd.read_parquet('/kaggle/input/otto-train-and-test-data-for-local-validation/test_labels.parquet')
click_tar = click_tar.loc[ click_tar['type']=='clicks' ]
click_tar = click_tar.explode('ground_truth')
click_tar = click_tar.rename(columns = {'ground_truth': 'item'})
click_tar['click'] = 1

In [27]:
click_tar['item'] = click_tar['item'].astype('int32')
click_tar

,session,type,item,click
0,11098528,clicks,1679529,1
3,11098529,clicks,1105029,1
6,11098532,clicks,1596491,1
7,11098533,clicks,1417450,1
10,11098534,clicks,908024,1
...,...,...,...,...
2212687,12899774,clicks,1399483,1
2212688,12899775,clicks,1760714,1
2212689,12899776,clicks,1737908,1
2212690,12899777,clicks,384045,1


In [28]:

click_candidates =pd.read_parquet(f'click_candidate_with_features_p0.pqt')
for k in range(1, 10):
    click_candidates = pd.concat([click_candidates, pd.read_parquet(f'click_candidate_with_features_p{k}.pqt')])

In [29]:
click_candidates

,session,aids,item_item_count,item_user_count,item_buy_ratio,user_user_count,user_item_count,user_buy_ratio,feature_click,feature_cart,feature_buy
0,11098528,11830,33776.0,19211.0,0.173970,1,1,0.0,1.0,0.0,0.0
1,11098528,588923,24588.0,14741.0,0.125346,1,1,0.0,0.0,0.0,0.0
2,11098528,1732105,10452.0,5770.0,0.280233,1,1,0.0,0.0,0.0,0.0
3,11098528,571762,18822.0,12405.0,0.124588,1,1,0.0,0.0,0.0,0.0
4,11098528,884502,31522.0,18163.0,0.135398,1,1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1511644,11198527,637827,12590.0,7183.0,0.140270,3,1,0.0,0.0,0.0,0.0
1511645,11198527,485256,36584.0,12763.0,0.214438,3,1,0.0,0.0,0.0,0.0
1511646,11198527,683401,6623.0,3234.0,0.116413,3,1,0.0,0.0,0.0,0.0
1511647,11198527,544144,37907.0,18053.0,0.238637,3,1,0.0,0.0,0.0,0.0


In [30]:
click_candidates['session'].nunique()

100000

In [31]:
click_candidates = click_candidates.merge(click_tar,left_on=['session','aids'],right_on = ['session', 'item'],how='left').fillna(0)
click_candidates = click_candidates.drop('item', axis=1)

In [32]:
click_candidates

,session,aids,item_item_count,item_user_count,item_buy_ratio,user_user_count,user_item_count,user_buy_ratio,feature_click,feature_cart,feature_buy,type,click
0,11098528,11830,33776.0,19211.0,0.173970,1,1,0.0,1.0,0.0,0.0,0,0.0
1,11098528,588923,24588.0,14741.0,0.125346,1,1,0.0,0.0,0.0,0.0,0,0.0
2,11098528,1732105,10452.0,5770.0,0.280233,1,1,0.0,0.0,0.0,0.0,0,0.0
3,11098528,571762,18822.0,12405.0,0.124588,1,1,0.0,0.0,0.0,0.0,0,0.0
4,11098528,884502,31522.0,18163.0,0.135398,1,1,0.0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14999995,11198527,637827,12590.0,7183.0,0.140270,3,1,0.0,0.0,0.0,0.0,0,0.0
14999996,11198527,485256,36584.0,12763.0,0.214438,3,1,0.0,0.0,0.0,0.0,0,0.0
14999997,11198527,683401,6623.0,3234.0,0.116413,3,1,0.0,0.0,0.0,0.0,0,0.0
14999998,11198527,544144,37907.0,18053.0,0.238637,3,1,0.0,0.0,0.0,0.0,0,0.0


In [33]:
click_candidates.dtypes

session              int32
aids                 int32
item_item_count    float64
item_user_count    float64
item_buy_ratio     float64
user_user_count      int64
user_item_count      int64
user_buy_ratio     float64
feature_click      float64
feature_cart       float64
feature_buy        float64
type                object
click              float64
dtype: object

In [34]:
click_candidates['aids'] = click_candidates['aids'].astype('int32')
click_candidates['item_item_count'] = click_candidates['item_item_count'].astype('int32')
click_candidates['item_user_count'] = click_candidates['item_user_count'].astype('int32')
click_candidates['item_buy_ratio'] = click_candidates['item_buy_ratio'].astype('float32')
click_candidates['user_user_count'] = click_candidates['user_user_count'].astype('int32')
click_candidates['user_item_count'] = click_candidates['user_item_count'].astype('int32')
click_candidates['user_buy_ratio'] = click_candidates['user_buy_ratio'].astype('float32')
click_candidates['feature_click'] = click_candidates['feature_click'].astype('int32')
click_candidates['feature_cart'] = click_candidates['feature_cart'].astype('int32')
click_candidates['feature_buy'] = click_candidates['feature_buy'].astype('int32')
click_candidates['click'] = click_candidates['click'].astype('int32')
click_candidates = click_candidates.drop('type', axis=1)

In [35]:
click_candidates.to_parquet('click_candidates.pqt')
click_candidates

,session,aids,item_item_count,item_user_count,item_buy_ratio,user_user_count,user_item_count,user_buy_ratio,feature_click,feature_cart,feature_buy,click
0,11098528,11830,33776,19211,0.173970,1,1,0.0,1,0,0,0
1,11098528,588923,24588,14741,0.125346,1,1,0.0,0,0,0,0
2,11098528,1732105,10452,5770,0.280233,1,1,0.0,0,0,0,0
3,11098528,571762,18822,12405,0.124588,1,1,0.0,0,0,0,0
4,11098528,884502,31522,18163,0.135397,1,1,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14999995,11198527,637827,12590,7183,0.140270,3,1,0.0,0,0,0,0
14999996,11198527,485256,36584,12763,0.214438,3,1,0.0,0,0,0,0
14999997,11198527,683401,6623,3234,0.116413,3,1,0.0,0,0,0,0
14999998,11198527,544144,37907,18053,0.238637,3,1,0.0,0,0,0,0


In [36]:
click_candidates[click_candidates['click'] == 1]

,session,aids,item_item_count,item_user_count,item_buy_ratio,user_user_count,user_item_count,user_buy_ratio,feature_click,feature_cart,feature_buy,click
150,11098529,1105029,206,149,0.033981,1,1,0.000000,1,0,0,1
900,11098534,908024,1168,825,0.020548,7,5,0.000000,1,0,0,1
1050,11098535,745365,75,57,0.040000,10,5,0.100000,1,1,0,1
1381,11098537,1503532,2028,1019,0.343688,23,17,0.304348,0,0,0,1
1514,11098538,1452081,1237,942,0.080032,29,25,0.103448,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
14998808,11198520,436475,3441,1891,0.076431,10,9,0.000000,1,0,0,1
14999100,11198522,676613,45,25,0.066667,5,5,0.000000,1,0,0,1
14999267,11198523,1492725,2209,1270,0.033952,1,1,0.000000,0,0,0,1
14999700,11198526,1035599,506,281,0.171937,2,1,0.000000,1,0,0,1
